In [1]:
import pymongo
import numpy as np
import pandas as pd
import datetime,time

In [5]:
# Connect with mongodb

try:
    with open("credentials.txt", 'r', encoding='utf-8') as f:
        [name,password,url,dbname]=f.read().splitlines()
        conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
    print ("Connected successfully!!!")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
    

    
db = conn['tweempact']

Connected successfully!!!


In [6]:
db.collection_names()

C:\Users\Albert Prat\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['system.indexes', 'tweets_noRT', 'users_noRT', 'tweets', 'tweet_users']

In [44]:
def read_mongo(collections, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    import pandas as pd
    import pymongo
    """ Read from Mongo and Store into DataFrame """
    # Connect with mongodb

    try:
        with open("credentials.txt", 'r', encoding='utf-8') as f:
            [name,password,url,dbname]=f.read().splitlines()
            conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
        print ("Connected successfully!!!")
    except pymongo.errors.ConnectionFailure as e:
        print ("Could not connect to MongoDB: %s" % e) 
     
    db = conn['tweempact']
    df1 = pd.DataFrame()
    for collection in collections:
        cursor = db[collection].find(query)
        df =  pd.DataFrame(list(cursor))
        if no_id:
            del df['_id']
        df1 = df1.append(df)
    df1 = df1.reset_index(drop = True)
    return df1

In [45]:
df = read_mongo(["tweets", "tweets_noRT"])

Connected successfully!!!


C:\Users\Albert Prat\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [46]:
len(df)

30651

In [47]:
 def author_dataset_conversion(df):   
    import time
    import numpy as np
    import pandas as pd

    cleaned = df[["id", "created_at","entities", "extended_entities","favorite_count","retweet_count","source","text", "truncated","user","quoted_status_id"]]
    ids = [user["id"] for user in cleaned["user"]]
    cleaned["id_num"] = ids

    #preparing output variables
    output = pd.DataFrame()
    users = []
    mean_RT_last10,mean_FC_last10 = [],[]
    sd_RT,sd_FC  = [],[]
    y1,y2=[],[]

    for _id in np.unique(ids):
        subset = cleaned[cleaned["id_num"]==_id].reset_index(drop=True)

        #filtering tweets from all tweets and RT
        l = list(subset["text"])
        filtr = ["RT" not in i for i in l]
        subset = subset[pd.Series(filtr)]
        if len(subset)>=11:
            reindexed = subset["created_at"].reset_index(drop=True)
            #convert tweet date into python date
            subset["created_at_pydate"] = [time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(reindexed[i],'%a %b %d %H:%M:%S +0000 %Y'))for i in range(0,len(reindexed))]
            subset = subset.sort_values(by="created_at_pydate").reset_index(drop=True)

            last10_tweets = subset[-11:-1]
            last_tweet = subset[-1:]
            #RT parameters last 10 tweets
            mean_RT_last10.append(np.mean(last10_tweets["retweet_count"]))
            sd_RT.append(np.std(last10_tweets["retweet_count"]))
            #FC parameters last 10 tweets
            mean_FC_last10.append(np.mean(last10_tweets["favorite_count"]))
            sd_FC.append(np.std(last10_tweets["favorite_count"]))
            #take json user
            j_user = subset[-1:]["user"].reset_index(drop=True)[0]
            users.append(j_user)
            #parameters of last tweet
            y1.append(np.mean(last_tweet["favorite_count"]))
            y2.append(np.mean(last_tweet["retweet_count"]))
    output["j_user"]=users
    output["RT_l10"]=mean_RT_last10
    output["sd_RT"]=sd_RT
    output["FC_l10"]=mean_FC_last10
    output["sd_FC"]=sd_FC
    output["FC"]=y1
    output["RT"]=y2
    return output

In [49]:
output2 = author_dataset_conversion(df)

C:\Users\Albert Prat\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [51]:
output2

,j_user,RT_l10,sd_RT,FC_l10,sd_FC,FC,RT
0,"{'id': 756431, 'id_str': '756431', 'name': 'Ar...",2.4,3.929377,15.5,32.022648,0.0,0.0
1,"{'id': 1622441, 'id_str': '1622441', 'name': '...",0.2,0.600000,0.8,0.979796,1.0,0.0
2,"{'id': 2107671, 'id_str': '2107671', 'name': '...",0.4,0.489898,1.7,3.796051,0.0,0.0
3,"{'id': 3127331, 'id_str': '3127331', 'name': '...",2.3,5.216321,3.4,6.545227,0.0,0.0
4,"{'id': 4869021, 'id_str': '4869021', 'name': '...",8.4,16.805951,76.1,127.668673,25.0,3.0
5,"{'id': 5254511, 'id_str': '5254511', 'name': '...",0.1,0.300000,0.4,0.489898,2.0,1.0
6,"{'id': 5380442, 'id_str': '5380442', 'name': '...",20.7,52.431002,49.7,98.777578,12.0,0.0
7,"{'id': 6709582, 'id_str': '6709582', 'name': '...",1.5,1.857418,14.1,21.215325,7.0,2.0
8,"{'id': 7996082, 'id_str': '7996082', 'name': '...",134.2,194.910646,147.3,214.264346,20.0,44.0
9,"{'id': 8932752, 'id_str': '8932752', 'name': '...",0.3,0.640312,1.3,2.368544,0.0,0.0


In [54]:
pd.read_csv(r"C:/Users/Albert Prat/Documents/tweempact/datasets/tweet.csv")

,Unnamed: 0,id,Tweet,j_user,created_at,entities,followers_user,FC,RT,trending_topic
0,0,1046429145914384386,Desde el #FCJumilla damos nuestra más sincera ...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:58:44,"{'hashtags': [{'text': 'FCJumilla', 'indices':...",7162,18,4,0.0
1,1,1044060298443919367,Me estoy volviendo loca hasta por cada una de ...,"{'id': 713359736, 'id_str': '713359736', 'name...",2018-09-24 03:05:47,"{'hashtags': [], 'symbols': [], 'user_mentions...",531,0,0,0.0
2,2,1046424001214648320,🏟 | El #FCJumilla ya está en La Condomina ⚽️...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:38:18,"{'hashtags': [{'text': 'FCJumilla', 'indices':...",7162,4,2,0.0
3,3,1046419267279818752,📹 | El vestuario del #FCJumilla en La Condomin...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:19:29,"{'hashtags': [{'text': 'FCJumilla', 'indices':...",7162,21,6,0.0
4,4,1046417005451128832,📑 Jornada 6 | El colegiado andaluz Guzmán Mans...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:10:30,"{'hashtags': [{'text': 'UCAMMurciaJumilla', 'i...",7162,0,0,0.0
5,5,1046416602739208193,🚐 | La expedición ya está lista para poner rum...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:08:54,"{'hashtags': [{'text': 'UCAMMurciaJumilla', 'i...",7162,10,2,0.0
6,6,1046343505197641728,⏱ | FINAL #FCJumillaB 1-1 @MulenoClubF ⚽️ ...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 10:18:26,"{'hashtags': [{'text': 'FCJumillaB', 'indices'...",7162,2,0,0.0
7,7,1046323550297239552,✋🏻⏱| DESCANSO #FCJumillaB 0-0 @MulenoClubF ...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 08:59:08,"{'hashtags': [{'text': 'FCJumillaB', 'indices'...",7162,0,0,0.0
8,8,1046300776455458816,🔊 VINÍCOLAS... 🔊 🔉 ¡¡El Filial también juega!...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 07:28:39,"{'hashtags': [], 'symbols': [], 'user_mentions...",7162,2,1,0.0
9,9,1045426450348888064,#CasadoDimisión,"{'id': 1401745850, 'id_str': '1401745850', 'na...",2018-09-27 21:34:23,"{'hashtags': [{'text': 'CasadoDimisión', 'indi...",329,0,0,0.0


In [52]:
pd.DataFrame.to_csv(output2, "C:/Users/Albert Prat/Documents/tweempact/datasets/author.csv")

In [53]:
def trending_topic_df(df, path="c:/Users/Albert Prat/chromedriver.exe", just_dates=False):    
    from selenium import webdriver
    import pandas as pd
    import numpy as np
    import datetime
    
    if just_dates==False:
    # preparing elements to iterate
        textdf = df["Tweet"]
        cadf = df["created_at"]
        cadf = [date[:10] for date in cadf]
        cadf = np.unique(cadf)
        cadf = [date.split("-") for date in cadf]
        iterable_for_adding_dummy = [date[2]+date[1]+date[0] for date in cadf]
        # to be able to work with <, >
    if just_dates == True:
        iterable_for_adding_dummy = df
    # setting the minimum date to look for and preparing the filters   
    u = [datetime.datetime.strptime(date, "%d%m%Y") for date in iterable_for_adding_dummy]
    threshold = datetime.datetime.strptime("01032015", "%d%m%Y")
    filtr = [u[i]>=threshold for i in range(len(u))]
    filtr1 = [u[i]<threshold for i in range(len(u))]
    
    iterable = np.array(iterable_for_adding_dummy)[filtr]

    # which of the unique dates is before threshold?
    indices = [i for i, x in enumerate(filtr1) if x == True]
    
    # entering to browser and select the date box to be inputted
    browser = webdriver.Chrome(path)
    browser.get("https://trendogate.com/")
    date_dd = browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/div[2]/input")
    
    # preparing output lists
    error = []
    grepl = []
    out = []
    for i in range(len(iterable)):
        browser.find_element_by_xpath("//select[@name='place']/option[text()='Spain']").click()
        date_dd = browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/div[2]/input")
        if just_dates == False:
            date = iterable[i][:10]
            date = date.split("-")
            date = date[2]+date[1]+date[0]
        if just_dates == True:
            date = iterable[i]
        date_dd.send_keys(date)
        browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/button").click()

        if len(browser.find_elements_by_class_name("alert")) == 1:
            error.append(date)
            out.append(None)
            pass
        else:
            b = browser.find_element_by_class_name("col-lg-8")
            lgi = b.find_elements_by_class_name("list-group-item")
            TT = [el.text for el in lgi]
            TT = ",".join(TT)
            out.append(TT)
    for index in indices:
        out.insert(index, None)
    dates = [date[:2]+"/"+date[2:4]+"/"+date[4:] for date in iterable_for_adding_dummy]
    tocsv = pd.DataFrame([dates, out]).T
    tocsv.columns=["date", "trending_topics"]
    return tocsv
def widen_TT_csv(df):
    old = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv")
    ca_old = old["created_at"]
    ca_old = [date[:10] for date in ca_old]
    ca_old = np.unique(ca_old)
    ca_old = [date.split("-") for date in ca_old]
    ca_old = [date[2]+date[1]+date[0] for date in ca_old]
    
    ca_new = df["created_at"]
    ca_new = [date[:10] for date in ca_new]
    ca_new = np.unique(ca_new)
    ca_new = [date.split("-") for date in ca_new]
    ca_new = [date[2]+date[1]+date[0] for date in ca_new]
    
    new_dates = set(ca_new) - set(ca_old)
    
    trend = trending_topic_df(list(new_dates), just_dates = True)
    output = pd.concat([old, trend], axis = 0).reset_index(drop = True)
    return output

In [54]:
def text_dataset(df):
    import re
    import pandas as pd
    l = list(df["text"])
    filtr = ["RT" not in i for i in l]
    df = df[filtr].reset_index(drop=True)
    
    Tweet =[re.sub("[(\n)(\r)]"," ",tweet) for tweet in df["text"]]
    entities = [tweet for tweet in df["entities"]]
    FC = [tweet for tweet in df["favorite_count"]]
    RT = [tweet for tweet in df["retweet_count"]]
    id_ = [tweet for tweet in df["id"]]
    j_user = [tweet for tweet in df["user"]]
    followers = [tweet["followers_count"] for tweet in df["user"]]
    
    CA = df["created_at"]
    CA_l = [time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(CA[i],'%a %b %d %H:%M:%S +0000 %Y'))for i in range(0,len(CA))]
    
    output = pd.DataFrame([id_,Tweet,j_user,CA_l,entities,followers, FC,RT]).T
    output.reset_index(drop=True)
    output.columns = ["id", "Tweet", "j_user","created_at", "entities", "followers_user", "FC","RT"]
    
    #trending topic df
    TT_new = widen_TT_csv(output)
    TT_new.to_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\trending_topics.csv")
    TT_new = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\trending_topics.csv")
    return output

In [55]:
output1 = text_dataset(df1)
output2 = text_dataset(df2)

In [62]:
#pd.DataFrame.to_csv(output2, "C:/Users/Albert Prat/Documents/tweempact/datasets/tweet.csv")

In [1051]:
tocsv = trending_topic_df(output2)

In [2]:
def widen_TT_csv(df):
    old = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv")
    ca_old = old["created_at"]
    ca_old = [date[:10] for date in ca_old]
    ca_old = np.unique(ca_old)
    ca_old = [date.split("-") for date in ca_old]
    ca_old = [date[2]+date[1]+date[0] for date in ca_old]
    
    ca_new = df["created_at"]
    ca_new = [date[:10] for date in ca_new]
    ca_new = np.unique(ca_new)
    ca_new = [date.split("-") for date in ca_new]
    ca_new = [date[2]+date[1]+date[0] for date in ca_new]
    
    new_dates = set(ca_new) - set(ca_old)
    
    trend = trending_topic_df(list(new_dates), just_dates = True)
    output = pd.concat([old, trend], axis = 0).reset_index(drop = True)
    return output

In [3]:
output.to_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\trending_topics.csv")

In [4]:
def append_dummy_TT(df):
    import re
    import unidecode
    import pandas as pd
    import ast
    
    textdf = df["Tweet"]
    trending_topics = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\trending_topics.csv")
    TT = trending_topics["trending_topics"]
    indices = TT[TT.isnull()].index
    # get TT as a list of lists
    out = []
    for i in range(len(TT)):
        if i in indices:
            out.append(None)
        else:
            t = TT[i].split(",")
            out.append(t)
    # date management
    ca = list(df["created_at"])
    ca = [date[:10] for date in ca]
    sp_d = [date.split("-") for date in ca]
    sp_d = [date[2]+date[1]+date[0] for date in sp_d]
    # just keeping unique dates
    ca = list(trending_topics["date"])
    unique_date = np.unique(ca)
    unique_date = [date.split("/") for date in unique_date]
    unique_date = [date[2]+date[1]+date[0] for date in unique_date]
    # append sp_d for the sake of indexing
    df["spanish_date"]=sp_d
    grepl = []
    for i in range(len(sp_d)):
        ls = []
        date = df[df.index == i]["spanish_date"][i]
        tweet = textdf[i].lower()
        tweet = unidecode.unidecode(tweet)
        index = np.where(trending_topics["date"]==(sp_d[i][:2]+"/"+sp_d[i][2:4]+"/"+sp_d[i][4:]))
        TT = out[index[0][0]]
        if TT == None:
            grepl.append(None)
        else:
            for word in TT:
                word = word.lower()
                word = unidecode.unidecode(word)
                word1 = re.sub(" ", "", word)
                ls.append(bool(re.search(r"\b"+str(word)+r"\b"+r"|\b"+str(word1)+r"\b", tweet)))
            grepl.append(np.sum(ls))
    df = df.drop("spanish_date", axis = 1)
    df["trending_topic"]=grepl
    df.reset_index(drop = True)
    
    df1 = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv", index_col=0)
    dropped = []
    for column in df.columns:
        dtype = type(df[column][0])
        if dtype == dict:
            df[column] = df[column].astype(str)
            dropped.append(column)
    df = pd.concat([df1,df], axis = 0).drop_duplicates().reset_index(drop=True)
    df.reset_index(drop = True)
    return df

In [493]:
out2 = append_dummy_TT(output2)

In [510]:
out2.to_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv")

In [511]:
out = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv", index_col=0)

In [514]:
len(out2)

42216

In [367]:
out.to_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv")